<a href="https://colab.research.google.com/github/DevLujain/LabWork/blob/main/CrabAgePrediciton.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
path = "/content/drive/MyDrive/Colab Notebooks/ModifiedCrabAgePrediction.csv"
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
# Load data set
data=pd.read_csv(path)

# Identify features and target variable
X = data.drop('Age', axis=1)
y = data['Age']

# Encoding categorical variable 'sex'
# assuming df is your DataFrame and 'Sex' is the column of mixed sex values

sex_dict = {'F': 0, 'M': 1, 'I': 2}

data['Sex'] = data['Sex'].map(sex_dict)

data.head()


,Sex,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight,Age
0,0,1.4375,1.1750,0.4125,24.635715,12.332033,5.584852,6.747181,9
1,1,0.8875,0.6500,0.2125,5.400580,2.296310,1.374951,1.559222,6
2,2,1.0375,0.7750,0.2500,7.952035,3.231843,1.601747,2.764076,6
3,0,1.1750,0.8875,0.2500,13.480187,4.748541,2.282135,5.244657,10
4,2,0.8875,0.6625,0.2125,6.903103,3.458639,1.488349,1.700970,6


In [ ]:
#To know the quantity of nulls in each column
data.isnull().sum()

Sex                0
Length            44
Diameter          32
Height            87
Weight            38
Shucked Weight     0
Viscera Weight     0
Shell Weight       0
Age                0
dtype: int64

In [ ]:
from sklearn.preprocessing import StandardScaler

columns_to_fill = ['Length', 'Diameter', 'Height', 'Weight']

# Fill missing values with mean
data[columns_to_fill] = data[columns_to_fill].fillna(data[columns_to_fill].mean())

# Scale or normalize numerical features
scaler = StandardScaler()
X = data.drop('Age', axis=1)  # Reassign X after filling missing values
X_scaled = scaler.fit_transform(X)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Apply Min-Max Scaling to numerical features
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
from scipy import stats

# Calculate Z-scores for X_scaled
z_scores = np.abs(stats.zscore(X_scaled))

# Define a threshold for outlier detection (e.g., Z-score < 3)
threshold = 3

# Find rows where all z-scores are less than the threshold
#This creates a boolean mask where each row will have a True only if all z-scores for that row are less than the threshold.
outlier_mask = (z_scores < threshold).all(axis=1)

# Filter X_scaled and y based on the outlier mask
X_scaled_no_outliers = X_scaled[outlier_mask]
y_no_outliers = y[outlier_mask]


In [ ]:
# Print shapes of original and outlier-removed datasets
print(f"Original dataset shape: {X_scaled.shape}")
print(f"Dataset shape after removing outliers: {X_scaled_no_outliers.shape}")
print(f"Original target variable shape: {y.shape}")
print(f"Target variable shape after removing outliers: {y_no_outliers.shape}")


Original dataset shape: (3500, 8)
Dataset shape after removing outliers: (3422, 8)
Original target variable shape: (3500,)
Target variable shape after removing outliers: (3422,)


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Define models with hyperparameters to tune
models = {
    'Linear Regression': LinearRegression(),
    'SVR': SVR(),
    'Gradient Boosting Regression': GradientBoostingRegressor(),
    'K-Nearest Neighbors Regression': KNeighborsRegressor(),
    'Decision Tree Regression': DecisionTreeRegressor(),
    'Random Forest Regression': RandomForestRegressor()
}

# Pipeline for Polynomial Regression
poly_reg = Pipeline([
    ('poly_features', PolynomialFeatures()),
    ('linear_reg', LinearRegression())
])

models['Polynomial Regression'] = poly_reg

# Initialize evaluation results dictionary
evaluation_results = {}

# Train, evaluate, and compare models
for model_name, model in models.items():
    if model_name == 'Polynomial Regression':
        param_grid = {
            'poly_features__degree': [2, 3, 4]
        }
        grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error')

        grid_search.fit(X_train, y_train)
        best_model = grid_search.best_estimator_
    else:
        model.fit(X_train, y_train)
        best_model = model

    y_pred = best_model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    evaluation_results[model_name] = {'MSE': mse, 'RMSE': rmse, 'R^2 Score': r2}
# Comparison
print("Model Evaluation Results:")
for model_name, metrics in evaluation_results.items():
    print(f"{model_name}:")
    for metric_name, value in metrics.items():
        print(f"  {metric_name}: {value:.2f}")



Model Evaluation Results:
Linear Regression:
  MSE: 5.40
  RMSE: 2.32
  R^2 Score: 0.45
SVR:
  MSE: 4.53
  RMSE: 2.13
  R^2 Score: 0.54
Gradient Boosting Regression:
  MSE: 4.54
  RMSE: 2.13
  R^2 Score: 0.54
K-Nearest Neighbors Regression:
  MSE: 5.02
  RMSE: 2.24
  R^2 Score: 0.49
Decision Tree Regression:
  MSE: 9.02
  RMSE: 3.00
  R^2 Score: 0.09
Random Forest Regression:
  MSE: 4.70
  RMSE: 2.17
  R^2 Score: 0.52
Polynomial Regression:
  MSE: 7.34
  RMSE: 2.71
  R^2 Score: 0.26


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression, Lasso, ElasticNet
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

# Define hyperparameter grids for the models
linear_param_grid = {}  # Linear Regression has no hyperparameters
svr_param_grid = {'C': [0.1, 1, 10], 'epsilon': [0.1, 0.01, 0.001]}
gb_param_grid = {'n_estimators': [50, 100, 200], 'max_depth': [3, 5, 7]}
knn_param_grid = {'n_neighbors': [3, 5, 7]}
dt_param_grid = {'max_depth': [3, 5, 7]}
rf_param_grid = {'n_estimators': [50, 100, 200], 'max_depth': [3, 5, 7]}

# Initialize the models
linear_model = LinearRegression()
svr_model = SVR()
gb_model = GradientBoostingRegressor()
knn_model = KNeighborsRegressor()
dt_model = DecisionTreeRegressor()
rf_model = RandomForestRegressor()

# Perform grid search for each model
models = {
    'Linear Regression': (linear_model, linear_param_grid),
    'SVR': (svr_model, svr_param_grid),
    'Gradient Boosting Regression': (gb_model, gb_param_grid),
    'K-nearest Neighbors Regression': (knn_model, knn_param_grid),
    'Decision Tree Regression': (dt_model, dt_param_grid),
    'Random Forest Regression': (rf_model, rf_param_grid)
}

results = {}

for model_name, (model, param_grid) in models.items():
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)
    grid_search.fit(X_train, y_train)

    best_model = grid_search.best_estimator_
    mse = mean_squared_error(y_test, best_model.predict(X_test))

    results[model_name] = {'Best Params': grid_search.best_params_, 'MSE': mse}

# Print the results
print("Model Evaluation Results:")
for model_name, metrics in results.items():
    print(f"{model_name}:")
    print(f"  Best Params: {metrics['Best Params']}")
    print(f"  MSE: {metrics['MSE']:.2f}")


Model Evaluation Results:
Linear Regression:
  Best Params: {}
  MSE: 5.40
SVR:
  Best Params: {'C': 10, 'epsilon': 0.1}
  MSE: 4.34
Gradient Boosting Regression:
  Best Params: {'max_depth': 3, 'n_estimators': 200}
  MSE: 4.54
K-nearest Neighbors Regression:
  Best Params: {'n_neighbors': 7}
  MSE: 4.74
Decision Tree Regression:
  Best Params: {'max_depth': 5}
  MSE: 5.30
Random Forest Regression:
  Best Params: {'max_depth': 7, 'n_estimators': 200}
  MSE: 4.59


In [ ]:
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression, Lasso, ElasticNet
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor

models = {
    'Linear Regression': LinearRegression(),
    'SVR': SVR(),
    'Gradient Boosting Regression': GradientBoostingRegressor(),
    'K-nearest Neighbors Regression': KNeighborsRegressor(),
    'Decision Tree Regression': DecisionTreeRegressor(),
    'Random Forest Regression': RandomForestRegressor()
}

r2_results = {}

for model_name, model in models.items():
    if model_name == 'Linear Regression':
        model = LinearRegression()
    elif model_name == 'SVR':
        model = SVR()
    elif model_name == 'Gradient Boosting Regression':
        model = GradientBoostingRegressor()
    elif model_name == 'K-nearest Neighbors Regression':
        model = KNeighborsRegressor()
    elif model_name == 'Decision Tree Regression':
        model = DecisionTreeRegressor()
    elif model_name == 'Random Forest Regression':
        model = RandomForestRegressor()

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    r2_results[model_name] = r2

# Print the R2 scores for each model
print("R2 Scores:")
for model_name, r2 in r2_results.items():
    print(f"{model_name}: {r2:.2f}")


R2 Scores:
Linear Regression: 0.45
SVR: 0.54
Gradient Boosting Regression: 0.54
K-nearest Neighbors Regression: 0.49
Decision Tree Regression: 0.08
Random Forest Regression: 0.53


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor

# Define hyperparameter grid for Linear Regression
linear_param_grid = {}  # Linear Regression has no hyperparameters

# Initialize models
models = {
    'Linear Regression': LinearRegression(),
    'SVR': SVR(),
    'Gradient Boosting Regression': GradientBoostingRegressor(),
    'K-nearest Neighbors Regression': KNeighborsRegressor(),
    'Decision Tree Regression': DecisionTreeRegressor(),
    'Random Forest Regression': RandomForestRegressor()
}

results = {}

for model_name, model in models.items():
    if model_name == 'Linear Regression':
        model = LinearRegression()
        param_grid = {}  # Linear Regression has no hyperparameters
    elif model_name == 'SVR':
        param_grid = {'C': [0.1, 1, 10], 'epsilon': [0.1, 0.01, 0.001]}
    elif model_name == 'Gradient Boosting Regression':
        param_grid = {'n_estimators': [50, 100, 200], 'max_depth': [3, 5, 7]}
    elif model_name == 'K-nearest Neighbors Regression':
        param_grid = {'n_neighbors': [3, 5, 7]}
    elif model_name == 'Decision Tree Regression':
        param_grid = {'max_depth': [3, 5, 7]}
    elif model_name == 'Random Forest Regression':
        param_grid = {'n_estimators': [50, 100, 200], 'max_depth': [3, 5, 7]}

    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)
    grid_search.fit(X_train, y_train)

    best_model = grid_search.best_estimator_
    mse = mean_squared_error(y_test, best_model.predict(X_test))
    r2 = r2_score(y_test, best_model.predict(X_test))

    results[model_name] = {'Best Params': grid_search.best_params_, 'MSE': mse, 'R2 Score': r2}

# Print the results
print("Model Evaluation Results:")
for model_name, metrics in results.items():
    print(f"{model_name}:")
    print(f"  Best Params: {metrics['Best Params']}")
    print(f"  MSE: {metrics['MSE']:.2f}")
    print(f"  R2 Score: {metrics['R2 Score']:.2f}")



Model Evaluation Results:
Linear Regression:
  Best Params: {}
  MSE: 5.40
  R2 Score: 0.45
SVR:
  Best Params: {'C': 10, 'epsilon': 0.1}
  MSE: 4.34
  R2 Score: 0.56
Gradient Boosting Regression:
  Best Params: {'max_depth': 3, 'n_estimators': 200}
  MSE: 4.60
  R2 Score: 0.53
K-nearest Neighbors Regression:
  Best Params: {'n_neighbors': 7}
  MSE: 4.74
  R2 Score: 0.52
Decision Tree Regression:
  Best Params: {'max_depth': 5}
  MSE: 5.30
  R2 Score: 0.46
Random Forest Regression:
  Best Params: {'max_depth': 7, 'n_estimators': 200}
  MSE: 4.58
  R2 Score: 0.54
